# Imports and Downloads

In [ ]:
import torch
from PIL import Image, ImageOps
import torch.utils.data as data
import torchvision.transforms as transforms
import six
import glob
import os
from os.path import join as pjoin
import pickle
from functools import reduce
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.nn import functional as F

In [ ]:
#%cd drive/My Drive/OpenFace
#!chmod 755 OpenFace/build/bin/FeatureExtraction

/content/drive/My Drive/OpenFace


In [ ]:
# RESNET DOWNLOAD + PYTORCH BENCHMARK

! git clone https://github.com/albanie/pytorch-benchmarks.git
! mkdir pytorch-benchmarks/ferplus/
# Resnet Model and Weights
! wget -P pytorch-benchmarks/ferplus/ http://www.robots.ox.ac.uk/~albanie/models/pytorch-mcn/resnet50_ferplus_dag.py
! wget -P pytorch-benchmarks/ferplus/ http://www.robots.ox.ac.uk/~albanie/models/pytorch-mcn/resnet50_ferplus_dag.pth 

fatal: destination path 'pytorch-benchmarks' already exists and is not an empty directory.
mkdir: cannot create directory ‘pytorch-benchmarks/ferplus/’: File exists
--2020-07-04 21:53:40--  http://www.robots.ox.ac.uk/~albanie/models/pytorch-mcn/resnet50_ferplus_dag.py
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52420 (51K) [text/x-python]
Saving to: ‘pytorch-benchmarks/ferplus/resnet50_ferplus_dag.py.1’

resnet50_ferplus_da 100%[===================>]  51.19K   324KB/s    in 0.2s    

2020-07-04 21:53:41 (324 KB/s) - ‘pytorch-benchmarks/ferplus/resnet50_ferplus_dag.py.1’ saved [52420/52420]

--2020-07-04 21:53:43--  http://www.robots.ox.ac.uk/~albanie/models/pytorch-mcn/resnet50_ferplus_dag.pth
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.

In [ ]:
# ANNOTATION FILE
%cp /content/drive/My\ Drive/Chalearn\ Data/train-annotation.zip /content 
!unzip "/content/train-annotation.zip"

def load_annotation(pickle_file):
    with open(pickle_file, 'rb') as f:
        pickle_data = pickle.load(f, encoding='latin1')
        df = pd.DataFrame(pickle_data)
        df.reset_index(inplace=True)
        del df['interview']
        df.columns = ["VideoName","ValueExtraversion","ValueNeuroticism","ValueAgreeableness","ValueConscientiousness","ValueOpenness"]
    return df

annotation_df = load_annotation("/content/annotation_training.pkl")
annotation_list = annotation_df.values.tolist()

Archive:  /content/train-annotation.zip
replace annotation_training.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
%cp -av /content/drive/My\ Drive/opface_output_dir /content/images_dir1

'/content/drive/My Drive/opface_output_dir' -> '/content/images_dir1/opface_output_dir'
cp: cannot create symbolic link '/content/images_dir1/opface_output_dir': Operation not supported


In [ ]:
video_list = glob.glob('/content/images_dir1/*.mp4')
print(len(video_list))

981


# Data Extraction; OpenFace

In [ ]:
class Video_Processor(object):
  def __init__(self, size=112, nomask=True, grey=False, quiet=True,tracked_vid=False, noface_save=False,OpenFace_exe = 'OpenFace/build/bin/FeatureExtraction'):

    self.size = size
    self.nomask = nomask
    self.grey = grey
    self.quiet = quiet
    self.tracked_vid = tracked_vid
    self.noface_save = noface_save
    self.OpenFace_exe = OpenFace_exe 
    
    print("Should be path to Open Face",self.OpenFace_exe)
  def process(self, input_video, output_dir=None):

      opface_option = " -f "+ input_video + " -out_dir "+ output_dir +" -simsize "+ str(self.size)
      opface_option += " -2Dfp -3Dfp -pdmparams -pose -aus -gaze -simalign "

      if not self.noface_save:
          opface_option +=" -nobadaligned "
      if self.tracked_vid:
          opface_option +=" -tracked "
      if self.nomask:
          opface_option+= " -nomask"
      if self.grey:
          opface_option += " -g"
      if self.quiet:
          opface_option += " -q"

      # Execution
      call = self.OpenFace_exe + opface_option
      os.system(call)


In [ ]:
# The First Called Class
class Tester(object):
  def __init__(self,
              # Parameters for testing
                batch_size,
                workers = 0,
                
                # Parameters for Video_Processor // All Features for OpenFace
                save_size=112, nomask=True, grey=False, quiet=True,
                tracked_vid=False, noface_save=False,
                OpenFace_exe = 'OpenFace/build/bin/FeatureExtraction',

                # Parameters for Snippet_Sampler
                num_phase=12, phase_size = 48, 
                length=64, stride=64,

                # Parameters for Phase_Difference_Extractor
                height=4, nbands=2, scale_factor=2, 
                extract_level = [1,2]
                ):

      # Calling Video Processor 
      self.video_processor = Video_Processor(save_size, nomask, grey, quiet,tracked_vid, noface_save, OpenFace_exe)
      #self.resnet50_extractor =  Resnet50_Extractor(benchmark_dir, model_name, feature_layer)
      #self.phase_difference_extractor = Phase_Difference_Extractor(height, nbands, scale_factor, extract_level, not quiet)

  def test(self, input_video):
  # Glob the path to Unzipped Videos  
    video_name = str(input_video.split('/')[-1])
    # Input video is processed using OpenFace # Change Path to your Open Face Directory
    !mkdir opface_output_dir/$video_name 
    # Calling Function Process (Video_Processor)
    opface_output_dir_path = 'opface_output_dir/'+video_name
    print(input_video, opface_output_dir_path)
    self.video_processor.process(input_video, opface_output_dir_path)


       

In [ ]:
# Saving all the files to Drive
import glob
import os
#tester = Tester( batch_size=64, workers=8, quiet=True)
#video_list = glob.glob('/content/drive/.shortcut-targets-by-id/114/Unzipped_Videos/*.mp4')
#for video_path in video_list[11:500]:
  #tester.test(video_path)

# Pre- Processing + Extra Functions

In [ ]:
"""
 Return:
        (transforms.Compose): Composition of preprocessing transform
# Transform Settings for the Pretrained Resnet Model
def compose_transforms(meta, resize=256, center_crop=True,
                       override_meta_imsize=False):
  
    normalize = transforms.Normalize(mean=meta['mean'], std=meta['std'])
    im_size = meta['imageSize']
    assert im_size[0] == im_size[1], 'expected square image size'
    
    # Transform 1 : Apply Center Crop
    if center_crop:
        transform_list = [transforms.Resize(resize),
                          transforms.CenterCrop(size=(im_size[0], im_size[1]))]

    else:
        if override_meta_imsize:
            im_size = (resize, resize)
        transform_list = [transforms.Resize(size=(im_size[0], im_size[1]))]
    transform_list += [transforms.ToTensor()]



    if meta['std'] == [1, 1, 1]:  # common amongst MCN models
        transform_list += [lambda x: x * 255.0]
    transform_list.append(normalize)

    return transforms.Compose(transform_list)
"""    

"\n Return:\n        (transforms.Compose): Composition of preprocessing transform\n# Transform Settings for the Pretrained Resnet Model\ndef compose_transforms(meta, resize=256, center_crop=True,\n                       override_meta_imsize=False):\n  \n    normalize = transforms.Normalize(mean=meta['mean'], std=meta['std'])\n    im_size = meta['imageSize']\n    assert im_size[0] == im_size[1], 'expected square image size'\n    \n    # Transform 1 : Apply Center Crop\n    if center_crop:\n        transform_list = [transforms.Resize(resize),\n                          transforms.CenterCrop(size=(im_size[0], im_size[1]))]\n\n    else:\n        if override_meta_imsize:\n            im_size = (resize, resize)\n        transform_list = [transforms.Resize(size=(im_size[0], im_size[1]))]\n    transform_list += [transforms.ToTensor()]\n\n\n\n    if meta['std'] == [1, 1, 1]:  # common amongst MCN models\n        transform_list += [lambda x: x * 255.0]\n    transform_list.append(normalize)\n\n  

In [ ]:
def load_model(model_name, MODEL_DIR):
    """
    Return:
        nn.Module: the loaded network
    """
    model_def_path = pjoin(MODEL_DIR, model_name + '.py')
    weights_path = pjoin(MODEL_DIR, model_name + '.pth')
    mod = load_module_2or3(model_name, model_def_path)
    func = getattr(mod, model_name)
    net = func(weights_path=weights_path)
    return net

def load_module_2or3(model_name, model_def_path):
    """Load model definition module in a manner that is compatible with
    both Python2 and Python3
    Args:
        model_name: The name of the model to be loaded
        model_def_path: The filepath of the module containing the definition
    Return:
        The loaded python module."""
    if six.PY3:
        import importlib.util
        spec = importlib.util.spec_from_file_location(model_name, model_def_path)
        mod = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(mod)
    else:
        import importlib
        dirname = os.path.dirname(model_def_path)
        sys.path.insert(0, dirname)
        module_name = os.path.splitext(os.path.basename(model_def_path))[0]
        mod = importlib.import_module(module_name)
    return mod    

# Resnet50 Feature Prep

## Video Recorder

In [ ]:
class VideoRecord(object):
  def __init__(self, video, face_dir, annot_dir, label_name, test_mode = False):
    '''        
    A VideoRecord that records all information of this video (and frames)
    '''
    self.video = video
    self.face_dir = face_dir
    self.annot_dir = annot_dir
    self.label_name = label_name 
    self.test_mode = test_mode
    self.path_label = self.get_path_label()
    print("path_label ka output", self.path_label)

  def get_path_label(self):
    '''        
    Return all the frames and labels of this video
    '''
    # splitting frames
    face_dir = self.face_dir
    video = face_dir.split('/')[-1] 
    to_add = video.split('.')
    video = to_add[0] + "." + to_add[1]


    frames = glob.glob(os.path.join(face_dir, video+"_aligned", '*.bmp'))
    frames = sorted(frames, key  = lambda x: os.path.basename(x).split(".")[0].split("_")[-1])
#print([frames, np.array([[-100]*5]*len(frames))])

#print(name)

    frames_list=[]
    labels_list=[]
    # Find The Labels for the video; assign to every frame
    for i in updated_annotation:
      label = []
      if (face_dir.split('/')[-1] == i[0]):
        label.append(i[1])
        label.append(i[2])
        label.append(i[3])
        label.append(i[4])
        label.append(i[5])
       
        # Every Other Frame till 35 frames; skipping the first 50
        # Try all Frames
        for idx,frame in enumerate(frames[50:120]):
          if idx%2 == 1:
            frames_list.append(frame)
            labels_list.append(label)  

    print(frames_list)
    print(labels_list)    
    return frames_list, labels_list
    

## Image Sampler

In [ ]:
class Image_Sampler(data.Dataset):
  def __init__(self, video_name, root_path, test_mode =False,  
      annot_dir=None, label_name=None, transform= None, verbose=False,
      size = 224):
      
      self.video_name = video_name
      self.root_path = root_path

      self.annot_dir = annot_dir      
      self.label_name = label_name
      self.test_mode = test_mode
      self.transform = transform
      print("Check if None or value", self.transform)
      self.size = size
      self.verbose = verbose
      self.parse_video()
      #self.frame_ids = None
      if self.transform is None:
          print("I will Create transforms")
          self._create_transform()
      assert self.transform is not None
      
  def parse_video(self):
      self.video_record = VideoRecord(self.video_name, self.root_path, 
          self.annot_dir, self.label_name, self.test_mode)
      frames, labels = self.video_record.path_label
      print("frames labels", frames, labels)

      if self.verbose:
          print("video {} has {} frames".format(self.video_name, len(frames)))
      self.frame_ids = np.arange(len(frames))
      print(self.frame_ids)

  def __len__(self):
        return len(self.frame_ids)

  def __getitem__(self, index):
        f_id = self.frame_ids[index]
        frames, labels = self.video_record.path_label
        frame, label = frames[f_id], labels[f_id]
        img = Image.open(frame)
        img = self.transform(img)
        return img, label, frame, self.video_record.video
  def _create_transform(self):
    if not self.test_mode:
        img_size = self.size
        resize = int(img_size * 1.2)
        transform_list = [transforms.Resize(resize),
                          transforms.RandomCrop(img_size),
                          transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
                          transforms.RandomHorizontalFlip(),
                          transforms.ToTensor(),
                          transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                std=[0.229, 0.224, 0.225]),
                        ]
    else:
        img_size = self.size
        transform_list = [transforms.Resize(img_size),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225]),
                        ]
    self.transform = transforms.Compose(transform_list)

# Resnet50 Extractor

In [ ]:
DEV_BATCH_SIZE = 64 # originally 16
DEV_NUM_WORKERS = 16 # originally 0

class Resnet50_Extractor(object):
  def __init__(self, benchmark_dir = 'pytorch-benchmarks',model_name= 'resnet50_ferplus_dag',
                feature_layer = 'pool5_7x7_s1'):
    
      ''' 
          Resnet50_Extractor: A feature extractor to extract the final convolutional layer's feature vector (2048 dimensional)
          and save those feature vectors to npy file in an 
          output directory.
      '''

      self.benchmark_dir = 'pytorch-benchmarks'
      self.model_name = model_name
      self.feature_layer = feature_layer

      
      # load resnet50 model
      model_dir = os.path.abspath(os.path.join(self.benchmark_dir, 'ferplus'))
      self.model = load_model(self.model_name, model_dir)
      self.model = self.model.to(device)
      #print(self.model)
      
      self.model.eval()
      #print("Prints the model only different from Train more inclined towards testing platform", self.model.eval())

      # load Tansformation Function
      meta = self.model.meta
      #self.transform = compose_transforms(meta, center_crop=True)
      self.transform = None
      #print(meta)
      #print(self.transform)


  def run(self, input_dir, output_dir, batch_size=DEV_BATCH_SIZE):
        '''        
        output_dir: string
            All extracted feature vectors will be stored in output directory.
        '''
        print(input_dir)
        video_name = os.path.basename(input_dir)
        print(video_name)

        # Transform jo conpose_transform se aya
        dataset = Image_Sampler(video_name, input_dir, test_mode = False, transform=self.transform)
        
        
        data_loader = torch.utils.data.DataLoader(
            dataset, 
            batch_size = batch_size, 
            shuffle=False, drop_last=False,
            num_workers=DEV_NUM_WORKERS, pin_memory=False )
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        else:
            if (len(os.listdir(output_dir)) != 0):
                if ('.npy' in os.listdir(output_dir)[0]):
                    print("output_dir {} already exists, feature extraction skipped.".format(output_dir))
                    return
        with torch.no_grad():
            for ims, target, img_path, video_name in tqdm(data_loader):
                ims = ims.to(device)
                output = self.get_vec(ims)
                for feature, path, video_n in zip(output, img_path, video_name):
                    des_path = os.path.join(output_dir, "%05d.npy"%self.get_frame_index(path))
                    np.save(des_path, feature)
        return

  def get_vec( self,  image):
        bs = image.size(0)
        layer = self.model._modules.get(self.feature_layer)
        my_embedding= torch.zeros([bs, 2048, 1, 1])
        def copy_data(m, i, o):
            my_embedding.copy_(o.data)
        h = layer.register_forward_hook(copy_data)
        h_x = self.model(image)
        h.remove()
        return F.relu(my_embedding.squeeze())
        
  def get_frame_index(self, frame_path):
        frame_name = frame_path.split('/')[-1]
        frame_num  = int(frame_name.split('.')[0].split('_')[-1])
        return frame_num      

# Snippet Sampler

In [ ]:
class Snippet_Sampler(data.Dataset):
    def __init__(self, video_name, root_path, feature_path,
                 annot_dir=None, label_name=None, test_mode =True, 
                 num_phase=12, phase_size = 48, 
                 length=64, stride=64,
                 verbose=False):
        '''Snippet_Sampler: An image sampler to sample snippets (a snippet consists of one RGB image and N
        greyscale images). The main inputs to it are two directories, one is the OpenFace processed faces directory,
        another is the directory containing renset50 feature vectors. 
        Parameters:
            video_name: string
                The processed video name
            root_path: string
                The directory path where cropped and aligned faces are stored (optionally other feature files).
            feature_path, string,
                The directory containing all renset50 feature vectors. 
            test_mode: bool, default False
                If True, it means the image sampler will only sample images, not annotations. Then annot_dir,
                label_name will not be used. And the labels will be dummy outputs.
                If Fae length of snippets returned.
            stride: int, default 32
                The stride taken when sampling sequence of snippets. If stride<length, it means 
                adjacent sequenlse, the image sampler will sample both images and annotations. Then annot_dir, label_name
                need to be specified.
            label_name: string, default None
                If test_mode is False, the label_name needs to be one of {'arousal', 'valence', 'arousal_valence'}
            num_phase: int, default 12
                number of phase difference images, input to the phase net of the mimamo net.
            phase_size: int, default 48
                phase image size, default is 48x48
            length: int, default 64
                Thce will overlap with each other.
        '''
        self.video_name = video_name
        self.root_path = root_path
        self.feature_path = feature_path
        self.annot_dir = annot_dir
        
        self.label_name = label_name
        self.test_mode = test_mode
        self.length = length 
        self.stride = stride 
        self.num_phase = num_phase
        self.phase_size = phase_size
        self.verbose = verbose
        self.parse_video()

    def parse_video(self):
        
        self.video_record = VideoRecord(self.video_name, self.feature_path, 
            self.annot_dir, self.label_name, self.test_mode)
        frames, labels = self.video_record.path_label
        self.seq_ranges = list()
        start, end = 0, self.length
        while end < len(frames) and (start<len(frames)):
            self.seq_ranges.append([start, end]) 
            start +=self.stride
            end = start+self.length

        if self.seq_ranges[-1][1] < len(frames):
            start = len(frames) - self.length
            end = len(frames)
            self.seq_ranges.append([start, end])  
        if self.verbose:        
            print("videos {}, number of seqs:{}".format(self.video_name, len(self.seq_ranges)))

    def __len__(self):
        return len(self.seq_ranges)
    def __getitem__(self, index):
        
        seq_ranges = self.seq_ranges[index]
        start, end = seq_ranges
        frames, labels = self.video_record.path_label
        seq_frames, seq_labels = frames[start:end], labels[start:end]
        # sample rgb images (features)
        imgs = []
        for f in seq_frames:
            imgs.append(np.load(f))
        # sample phase images 
        # figure ids
        sample_f_ids = []
        for f_id in range(start, end):
            phase_ids = []
            for i in range(self.num_phase+1):
                step = i-self.num_phase//2
                id_0 = max(0,f_id + step)
                id_0 = min(id_0, len(frames)-1) 
                phase_ids.append(id_0)
            sample_f_ids.append(phase_ids)
        sample_frames = [[frames[id] for id in ids] for ids in sample_f_ids]
        # load greyscale images
        phase_images= []
        for frames in sample_frames:
            phase_img_list = []
            for frame in frames:
                f_index = int(os.path.basename(frame).split(".")[0])
                img_frame = os.path.join(self.root_path, self.video_record.video+"_aligned", 
                    'frame_det_00_{:06d}.bmp'.format(f_index))
                try:
                   img = Image.open(img_frame).convert('L')
                except:
                    raise ValueError("incorrect face path")    
                phase_img_list.append(img)
            phase_images.append(phase_img_list)
        if not self.test_mode:
            random_seed = np.random.randint(250)
            phase_transform = torchvision.transforms.Compose(
                                   [GroupRandomHorizontalFlip(seed=random_seed),
                                   GroupRandomCrop(size=int(self.phase_size*0.85), seed=random_seed),
                                   GroupScale(size=self.phase_size),
                                   Stack(),
                                   ToTorchFormatTensor()])
        else:
            phase_transform = torchvision.transforms.Compose([
                                   GroupScale(size=self.phase_size),
                                   Stack(),
                                   ToTorchFormatTensor()]) 
        flat_phase_images = []
        for sublist in phase_images:
            flat_phase_images.extend(sublist)
        flat_phase_images = phase_transform(flat_phase_images)
        phase_images = flat_phase_images.view(len(phase_images), self.num_phase+1, self.phase_size, self.phase_size)
        return phase_images, np.array(imgs), np.array(seq_labels), np.array([start, end]), self.video_record.video
    

## TRAIN

# if __name__ == "__main__":

#     from dataloader import Face_Dataset
#     import os
#     
#     train_dataset = Face_Dataset(root_path, feature_path, annot_dir, video_names, label_name='arousal', test_mode=False, num_phase=12, length=64, stride=32)
#     train_loader = torch.utils.data.DataLoader(
#         train_dataset, 
#         batch_size = 2, 
#         num_workers=0, pin_memory=False )
#     model = Two_Stream_RNN(mlp_hidden_units=[2048, 256, 256])
#     model.cuda()
#     model.train()
#     for phase_f, rgb_f, labels, ranges, videos in train_loader:
#         phase_0, phase_1 = phase_f
#         phase_0 = phase_0.type('torch.FloatTensor').cuda()
#         phase_1 = phase_1.type('torch.FloatTensor').cuda()
#         rgb_f = rgb_f.type('torch.FloatTensor').cuda()
#         labels = labels.type('torch.FloatTensor').cuda()
#         out = model([phase_0,phase_1], rgb_f)

# My 3 Models

## Fully Connected Layers

In [ ]:
class MLP(nn.Module):
    def __init__(self, hidden_units, dropout=0.3):
        super(MLP, self).__init__()
        input_feature_dim = hidden_units[0]
        num_layers = len(hidden_units)-1
        assert num_layers>0
        assert hidden_units[-1]==256
        fc_list = []
        for hidden_dim in hidden_units[1:]:
            fc_list += [ nn.Dropout(dropout),
                        nn.Linear(input_feature_dim, hidden_dim),
                        nn.BatchNorm1d(hidden_dim),
                        nn.ReLU(inplace=True)
                        ]
            input_feature_dim = hidden_dim
        self.mlp = nn.Sequential(*fc_list)
    def forward(self, input_tensor):
        bs, num_frames, feature_dim = input_tensor.size()
        input_tensor = input_tensor.view(bs*num_frames, feature_dim)
        out = self.mlp(input_tensor)
        return out.view(bs, num_frames, -1)

NameError: ignored

## PhaseNet

In [ ]:
class PhaseNet(nn.Module):
    def __init__(self, input_size, num_channels, hidden_units=[256, 256, 1] , dropout=0.3, feature=False):
        super(PhaseNet,self).__init__()
        # input size : 2**i times 6 or 7
        if input_size not in [48, 96, 112]:
               raise ValueError("Incorrect input size")
        if input_size==48:
            num_conv_layers = 3
        else:
            num_conv_layers = 4
        if input_size==48 or input_size==96:
            last_conv_width = 6
        else:
            last_conv_width = 7
        self.conv_net = []
        for i in range(num_conv_layers):
            if i==0:
                self.conv_net.append(self._make_conv_layer(num_channels, 2**(i+6), kernel_size=3, stride=2))
            elif i==1:
                self.conv_net.append(self._make_conv_layer(num_channels+2**(i-1+6), 2**(i+6), kernel_size=3, stride=2))
            else:
                self.conv_net.append(self._make_conv_layer(2**(i-1+6), 2**(i+6), kernel_size=3, stride=2))
        last_conv_dim = 2**(i+6)
        self.conv_net = nn.ModuleList(self.conv_net)
        self.dropout = nn.Dropout2d(p=0.2)
        self.avgpool = nn.AvgPool2d(kernel_size=[last_conv_width, last_conv_width])
        fc_list =[]
        fc_list += [nn.Linear(last_conv_dim, hidden_units[0]),
                       nn.ReLU(inplace=True),
                       nn.BatchNorm1d(hidden_units[1]),
                       nn.Dropout(dropout)]
        for i in range(0, len(hidden_units)-2):
            fc_list += [nn.Linear(hidden_units[i], hidden_units[i+1]),
                       nn.ReLU(inplace=True),
                       nn.BatchNorm1d(hidden_units[i+1]),
                       nn.Dropout(dropout)]
        self.fc = nn.Sequential(*fc_list)
        final_norm = nn.BatchNorm1d(1, eps=1e-6, momentum=0.1) 
        self.classifier = nn.Sequential(nn.Linear(hidden_units[-2], hidden_units[-1]),
                                 final_norm )
        self.feature = feature
    def _make_conv_layer(self, in_c, out_c, kernel_size = 3, stride = 2):
        ks = kernel_size 
        conv_layer = nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=(ks, ks), padding=ks//2),
        nn.BatchNorm2d(out_c, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_c, out_c, kernel_size=(ks, ks), padding=ks//2,stride=stride),
        nn.BatchNorm2d(out_c, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        nn.ReLU(inplace=True),
        )
        return conv_layer
    def forward(self, data_level0, data_level1):
        bs, num_frames,  num_channel, W0, H0 = data_level0.size()
        bs, num_frames, num_channel,  W1, H1 = data_level1.size()
        trans_data_level0 = data_level0.view(bs*num_frames, num_channel, W0, H0 )
        trans_data_level1 = data_level1.view(bs* num_frames, num_channel,  W1, H1)
        conv1 = self.conv_net[0](trans_data_level0)
        conv_out = torch.cat([conv1, trans_data_level1], dim=1) 
        for layer in self.conv_net[1:]:
            conv_out = self.dropout(layer(conv_out))
        avgpool = self.avgpool(conv_out)
        avgpool = avgpool.view(bs*num_frames, -1)
        out = self.fc(avgpool)
        if self.feature:
            return out
        else:
            out = self.classifier(out)
            return out

## Two Stream RNN

In [ ]:
class Two_Stream_RNN(nn.Module):
    def __init__(self, mlp_hidden_units=[2048, 256, 256], dropout=0.5, label_name = 'arousal_valence', 
        num_phase=12):
        super(Two_Stream_RNN, self).__init__()
        '''    
        Definition of the MIMAMO Net, consisting of a spatial stream and a temperal stream, followed
         by a RNN.
        Parameters: 
            mlp_hidden_units: list of int, default is [2048, 256, 256]
                The MLP defined to transform the Resnet50 feature from 2048 dimensions to 256 dimensions
            label_name: string, one of {'arousal', 'valence', 'arousal_valence'}
                The label name decides the output layer dimensions
            num_phase: int, default 12
                number of phase difference images, input to the phase net.
        '''
        
        self.mlp = MLP(mlp_hidden_units)
        self.num_phase = num_phase
        self.phasenet = PhaseNet(48, 2*num_phase, hidden_units =[256, 256, 1], dropout=0.3, feature=True)
        self.transform = nn.Sequential(nn.Linear(512, 256),
                                      nn.ReLU(inplace=True),
                                      nn.BatchNorm1d(256),
                                      nn.Dropout(dropout))
        self.rnns = nn.GRU(256, 128, bidirectional=True, num_layers=2, dropout = 0.3)
        self.classifier = nn.Sequential(nn.Dropout(dropout),
                                        nn.Linear(256, len(label_name.split("_"))),
                                        nn.BatchNorm1d(len(label_name.split("_"))))
    def load_model_weights(self, model, model_path):
        ckp = torch.load(model_path)
        net_key = [key for key in ckp.keys() if (key !='epoch') and (key !='iter')][0]
        state_dict = ckp[net_key]
        model.load_state_dict(state_dict)
        return model
    def forward(self, phase_data, rgb_data):
        bs, num_frames = rgb_data.size(0), rgb_data.size(1)
        features_cnn = self.mlp(rgb_data)
        features_spatial = features_cnn.view(bs, num_frames, -1)
        phase_0, phase_1 = phase_data
        features_temporal = self.phasenet(phase_0, phase_1)
        features_temporal = features_temporal.view(bs, num_frames, -1)
        features = torch.cat([features_spatial, features_temporal], dim=-1)
        features = self.transform(features.view(bs*num_frames, -1))
        features = features.view(bs, num_frames, -1)
        outputs_rnns,  _ = self.rnns(features)
        outputs_rnns = outputs_rnns.view(bs* num_frames, -1)
        out = self.classifier(outputs_rnns)
        out = out.view(bs, num_frames, -1)
        return out

# Train 

In [ ]:
class Trainer(object):
  def __init__(self,
                # Parameters for Resnet50_Extractor
                  benchmark_dir = 'pytorch-benchmarks',model_name= 'resnet50_ferplus_dag',
                  feature_layer = 'pool5_7x7_s1',
                
                  # Parameters for Snippet_Sampler
                  num_phase=12, phase_size = 48, 
                  length=64, stride=64,):
    
    self.resnet50_extractor =  Resnet50_Extractor(benchmark_dir, model_name, feature_layer)
    #self.phase_difference_extractor = Phase_Difference_Extractor(height, nbands, scale_factor, extract_level, not quiet)
    #self.model = Two_Stream_RNN()
  
  def train(self, video_name):
    #The Center Cropped Resized to 224 x 224 and aligned faces are fed to resnet50_extractor
    %mkdir "/content/resnet_features"
    feature_dir = os.path.join(os.path.dirname("/content/resnet_features"), video_name+"_pool5")
    self.resnet50_extractor.run(os.path.join(os.path.dirname("/content/images_dir1/"),video_name), feature_dir)
    
    
    # Sampling images
    dataset = Snippet_Sampler(video_name, opface_output_dir, feature_dir,
      annot_dir = None, label_name = 'valence_arousal',
        test_mode = False, num_phase=self.num_phase, phase_size = self.phase_size, 
        length=self.length, stride=self.stride)
'''    
    data_loader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=self.batch_size,
                    num_workers=self.workers, pin_memory=False)
    results = self.test_on_dataloader(data_loader, self.model)
    return results  


 def train_on_dataloader(self, dataloader, model, train_mean=None, train_std=None):
        model.eval()
        sample_names = []
        sample_preds = []
        sample_ranges = []
        for i, data_batch in enumerate(dataloader):
            phase_f, rgb_f, label, ranges, names = data_batch
            with torch.no_grad():
                phase_f = phase_f.type('torch.FloatTensor').to(device)
                phase_0, phase_1 = self.phase_diff_output(phase_f, self.phase_difference_extractor)
                rgb_f = Variable(rgb_f.type('torch.FloatTensor').to(device))
                phase_0 = Variable(phase_0.type('torch.FloatTensor').to(device))
                phase_1 = Variable(phase_1.type('torch.FloatTensor').to(device))
            
            output = model([phase_0,phase_1], rgb_f)
            sample_names.append(names)
            sample_ranges.append(ranges)
            sample_preds.append(output.cpu().data.numpy())
        sample_names = np.concatenate([arr for arr in sample_names], axis=0)
        sample_preds = np.concatenate([arr for arr in sample_preds], axis=0)
        n_sample, n_length, n_labels = sample_preds.shape
        if train_mean is not None and train_std is not None:
            # scale 
            trans_sample_preds = sample_preds.reshape(-1, n_labels)
            trans_sample_preds = np.array([correct(trans_sample_preds[:, i], train_mean[i], train_std[i]) for i  in range(n_labels)])
            sample_preds = trans_sample_preds.reshape(n_sample, n_length, n_labels)
        sample_ranges = np.concatenate([arr for arr in sample_ranges], axis=0)
        video_dict = {}
        for video in sample_names:
            mask = sample_names==video
            video_ranges = sample_ranges[mask]
            if video not in video_dict.keys():
                max_len = max([ranges[-1] for ranges in video_ranges])
                video_dict[video] = np.zeros((max_len, n_labels))
            video_preds = sample_preds[mask]
            # make sure the dataset returns full range of video frames
            min_f, max_f = 0, 0
            for rg, pred in zip(video_ranges, video_preds):
                start, end = rg
                video_dict[video][start:end, :] = pred
          
      min_f = min(min_f, start)
                max_f = max(max_f, end)
            assert (min_f==0) and (max_f == max_len)
        for video in video_dict.keys():
            video_dict[video] = pd.DataFrame(data=video_dict[video], columns=self.label_name)
        return video_dict  
'''        

"\n def train_on_dataloader(self, dataloader, model, train_mean=None, train_std=None):\n        model.eval()\n        sample_names = []\n        sample_preds = []\n        sample_ranges = []\n        for i, data_batch in enumerate(dataloader):\n            phase_f, rgb_f, label, ranges, names = data_batch\n            with torch.no_grad():\n                phase_f = phase_f.type('torch.FloatTensor').to(device)\n                phase_0, phase_1 = self.phase_diff_output(phase_f, self.phase_difference_extractor)\n                rgb_f = Variable(rgb_f.type('torch.FloatTensor').to(device))\n                phase_0 = Variable(phase_0.type('torch.FloatTensor').to(device))\n                phase_1 = Variable(phase_1.type('torch.FloatTensor').to(device))\n            \n            output = model([phase_0,phase_1], rgb_f)\n            sample_names.append(names)\n            sample_ranges.append(ranges)\n            sample_preds.append(output.cpu().data.numpy())\n        sample_names = np.concat

In [ ]:
def update_annotation():
  count = 0
  updated_annotation = []
  for video in video_list:
    for annot in annotation_list:
      if (video.split('/')[-1] == annot[0]):
        updated_annotation.append(annot)
  return updated_annotation        

In [ ]:
updated_annotation = update_annotation()
print(updated_annotation)

[['P63CUj9KnSw.001.mp4', 0.3271028037383178, 0.32291666666666663, 0.4395604395604395, 0.4466019417475728, 0.4], ['a97PXgrKSxo.003.mp4', 0.5233644859813085, 0.6354166666666666, 0.7582417582417583, 0.6699029126213591, 0.611111111111111], ['SxRNdq2nGN0.001.mp4', 0.38317757009345793, 0.53125, 0.4725274725274725, 0.42718446601941745, 0.5666666666666667], ['HL-K_PZXFR4.005.mp4', 0.7289719626168225, 0.6979166666666666, 0.7912087912087912, 0.8058252427184466, 0.5444444444444444], ['dLaqub4BEsA.002.mp4', 0.5607476635514019, 0.5520833333333334, 0.6813186813186812, 0.5242718446601942, 0.611111111111111], ['Vy-Lr_iM5qo.001.mp4', 0.47663551401869164, 0.3854166666666667, 0.4505494505494505, 0.4563106796116505, 0.5111111111111111], ['f39E4ct09Cc.005.mp4', 0.6168224299065421, 0.7083333333333334, 0.6153846153846153, 0.6796116504854369, 0.8222222222222221], ['Tg8a_iNEamM.005.mp4', 0.28037383177570097, 0.29166666666666663, 0.2747252747252747, 0.36893203883495146, 0.3666666666666667], ['X1EjfXX3NTk.003.mp

#FINAL

In [ ]:
# Find the videos that were not copied to Drive
video_list = glob.glob('/content/drive/.shortcut-targets-by-id/116/Unzipped_Videos/*.mp4')
str1 = "huIQlWsIEvY.003.mp4"
for idx, i in enumerate(video_list):
  if i.split('/')[-1] in a&b:
    print(idx)

In [ ]:
device = 'cuda:0'
assert isinstance(device, str)
num_cuda = torch.cuda.device_count()
if 'cuda' in device:
        if num_cuda > 0:
            # Found CUDA device, use the GPU
            print(num_cuda)
            print(torch.device(device))
            


1
cuda:0


In [ ]:
trainer = Trainer()
video_name = video_list[1].split('/')[-1]
trainer.train(video_name)


mkdir: cannot create directory ‘/content/resnet_features’: File exists







  0%|          | 0/1 [00:00<?, ?it/s]

/content/images_dir1/a97PXgrKSxo.003.mp4
a97PXgrKSxo.003.mp4
Check if None or value None
['/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000052.bmp', '/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000054.bmp', '/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000056.bmp', '/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000058.bmp', '/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000060.bmp', '/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000062.bmp', '/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000064.bmp', '/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000066.bmp', '/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000068.bmp', '/content/images_dir1/a97PXgrKSxo.003.mp4/a97PXgrKSxo.003_aligned/frame_det_00_000






100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


In [ ]:
file_check_resnet = open("/content/a97PXgrKSxo.003.mp4_pool5/00054.npy",'rb')


TypeError: ignored